# Drug Pricing Project - Descriptive Statistics

Import Packages and define arguments

In [1]:
import csv
import re
import os
import nltk
import argparse
import itertools
import glob
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict, deque, OrderedDict
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from itertools import chain
pd.options.display.max_colwidth = 450
porter_stemmer = PorterStemmer()

In [2]:
import sys
sys.path.insert(0, '/Users/jackiereimer/Dropbox/Drug Pricing Project/code/reddit_preprocessing')
from reddit_preprocessing import reddit_preprocessing as rp

In [3]:
class ArgumentContainer(object):
    def __init__(self):
        self.data_folder = "opiates"
        self.keyterm_folder = "keyterm_lists"
        self.complete_threads_file = "use_data/threads/all_dumps.csv"
        self.complete_comments_file = "use_data/comments/all_comments.csv"
        self.stop_words = "stop_words"
        self.location_folder = "location"
        self.mat_folder = "mat"
        self.unit_folder = "unit"
        self.currency_folder = "currency"
        self.output_folder = "output"
        self.file_folder = None


if 'args' not in dir():
    args = ArgumentContainer()

### Set filepaths and lists of keywords

In [6]:
locations_filepath, mat_filepath, all_comments_filepath, all_dumps_filepath, unit_filepath, currency_filepath, output_filepath, stopwords_filepath = rp.assign_location_dirs(args.data_folder, args.complete_threads_file, args.complete_comments_file, args.location_folder, args.mat_folder, args.unit_folder, args.currency_folder, args.output_folder, args.stop_words, args.file_folder)
state_init, locations = rp.generates_non_case_sensitive_list_of_keyterms(locations_filepath)
meth_words, sub_words, nalt_words, narc_words = rp.generates_non_case_sensitive_list_of_keyterms(mat_filepath)
currencies = rp.generates_non_case_sensitive_list_of_keyterms(currency_filepath)[0]
units = rp.generates_non_case_sensitive_list_of_keyterms(unit_filepath)[0]
more_stops = rp.generates_non_case_sensitive_list_of_keyterms(stopwords_filepath)[0]

All thread file: /Users/jackiereimer/Dropbox/drug_pricing_data/opiates/use_data/threads/all_dumps.csv
All comment file: /Users/jackiereimer/Dropbox/drug_pricing_data/opiates/use_data/comments/all_comments.csv
Stop Words file: /Users/jackiereimer/Dropbox/Drug Pricing Project/keyterm_lists/stop_words
Locations file: /Users/jackiereimer/Dropbox/Drug Pricing Project/keyterm_lists/location
MAT file: /Users/jackiereimer/Dropbox/Drug Pricing Project/keyterm_lists/mat
Unit file: /Users/jackiereimer/Dropbox/Drug Pricing Project/keyterm_lists/unit
Currency file: /Users/jackiereimer/Dropbox/Drug Pricing Project/keyterm_lists/currency
Output folder: /Users/jackiereimer/Dropbox/Drug Pricing Project/keyterm_lists/output
/Users/jackiereimer/Dropbox/Drug Pricing Project/keyterm_lists/location/state_init.csv
/Users/jackiereimer/Dropbox/Drug Pricing Project/keyterm_lists/location/locations.csv
/Users/jackiereimer/Dropbox/Drug Pricing Project/keyterm_lists/mat/naltrexone_words.csv
/Users/jackiereimer/Dro

There are 774 items within locations. After running the cell above, check the length to make sure that this is the case. If it is not, switch state_init and locations above and run again. This is the result of poor filepath generation that is not a big enough issue to warrant the amount of time that it would take to correct it. 

In [7]:
len(locations)

774

In [8]:
total_thread_tuples, total_threads = rp.list_of_threads_from_csv(args.data_folder, all_dumps_filepath)
total_comment_tuples, total_comments = rp.list_of_comments_from_csv(args.data_folder, all_comments_filepath)
total_posts = total_threads + total_comments
thread_tuples_headers = ['post_id','time','no_comments', 'post_title', 'post_body']
comment_tuples_headers = ['comment_id', 'time', 'reply_id', 'post_body']
stop = stopwords.words('english')
stop_all = stop + more_stops

All r/opiates/ threads aggregated
All r/opiates/ comments aggregated


## Put Data into DataFrame and Define Regex Filters

In [9]:
general_re = r"^.*\b({})\b.*$" # matches standalone strings
digit_re = r"\s\b\d{1,3}\b" # matches standalone numbers between 1 and 3 digits
price_re = r'^.*[{}]\s?\d{{1,3}}(?:[.,]\d{{3}})*(?:[.,]\d{{1,2}})?.*$' # matches standalone numbers of currency format with preceding currency symbol
unit_price_re = r'[{}]?\d+[/]\D\S+' # matches string of format 'digit(s)/letter(s)' (e.g. $40/gram, 5/mg)
surrounding_dollar_re = r'(?P<before>(?:\w+\W+){5})\$\d+(?:\.\d+)?(?P<after>(?:\W+\w+){5})' # matches the five words that surround the mention of '$'
surrounding_words_re = r'(?P<before>(?:\w+\W+){})[{}]\d+(?:\.\d+)?(?P<after>(?:\W+\w+){})' # requires three inputs (digit, keywords, digit), matches the digit number of words that surround keyword

In [10]:
def convert_key_word_threads_to_df(df, search_for, regexp, case_sensitive=False):
    """
    """
    print('Number of strings searched: %s' % df.shape[0])
    print('Number of keywords searching for: %s' % len(search_for))
    dt_start = datetime.datetime.now()
    print('Starting time:', dt_start)
    if not case_sensitive:
        flag = re.I
    else:
        flag = False
    i = 0
    new_df = df
    for keyword in search_for:
        i += 1
        print('Word %s out of %s' % (i, len(search_for)))
        print('Time elapsed:', datetime.datetime.now() - dt_start)
        word = re.compile(regexp.format(keyword), flags=flag)
        new_df[keyword] = new_df.astype(str).sum(axis=1).str.contains(word, regex=True)
    return new_df

In [11]:
comment_df = pd.DataFrame(total_comment_tuples, columns=comment_tuples_headers)
thread_df = pd.DataFrame(total_thread_tuples, columns=thread_tuples_headers)
reddit_df = thread_df.append(comment_df)
reddit_df = reddit_df[['time', 'post_id', 'reply_id', 'no_comments', 'comment_id', 'post_title', 'post_body']]
#reddit_df = reddit_df.applymap(lambda s:s.lower() if type(s) == str else s)

/Users/jackiereimer/beautifulsoup/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


## Descriptive Stats

In [48]:
comment_df['time_new'] = pd.to_datetime(comment_df['time'], unit='s')
comment_df['date'] = comment_df['time_new'].dt.date
comment_df['time_new'] = pd.to_datetime(comment_df['time'], unit='s')
comment_df['time'] = pd.to_numeric(comment_df['time'])
comment_df['date'] = pd.to_datetime(comment_df['time_new'])

thread_df['time_new'] = pd.to_datetime(thread_df['time'], unit='s')
thread_df['date'] = thread_df['time_new'].dt.date
thread_df['time_new'] = pd.to_datetime(thread_df['time'], unit='s')
thread_df['time'] = pd.to_numeric(thread_df['time'])
thread_df['date'] = pd.to_datetime(thread_df['time_new'])

In [66]:
comment_df.time = pd.to_datetime(comment_df.time, unit='s')
thread_df.time = pd.to_datetime(thread_df.time, unit='s')
#year_count = pd.DataFrame(thread_df['time'].resample('Y').count())
comment_count = pd.DataFrame(comment_df['comment_id'].resample('Y').count(), columns = year_count_headers)
#result = pd.merge(year_count, comment_count, on='time')

In [94]:
comment_year_count = pd.DataFrame(comment_df.set_index('time').resample('Y').size())
comment_year_count.rename_axis('Year')

thread_year_count = pd.DataFrame(thread_df.set_index('time').resample('Y').size())
thread_year_count.rename_axis('Year')

result = pd.merge(thread_year_count, comment_year_count, on='time')
mapping = {result.columns[0]:'Threads', result.columns[1]: 'Comments'}
result['Total'] = result.sum(axis=1)
result.loc['Total',:]= result.sum(axis=0)
result.rename(columns=mapping)

,Threads,Comments,Total
time,,,
2010-12-31 00:00:00,42.0,359.0,401.0
2011-12-31 00:00:00,270.0,3705.0,3975.0
2012-12-31 00:00:00,3403.0,63412.0,66815.0
2013-12-31 00:00:00,7656.0,150472.0,158128.0
2014-12-31 00:00:00,11450.0,207922.0,219372.0
2015-12-31 00:00:00,13700.0,249304.0,263004.0
2016-12-31 00:00:00,18834.0,331180.0,350014.0
2017-12-31 00:00:00,32056.0,471394.0,503450.0
2018-12-31 00:00:00,65227.0,358684.0,423911.0
